In [1]:
import pandas as pd
import fnmatch
import dask.dataframe as dd
from intake.source.utils import reverse_format
import os
import re
import subprocess
from tqdm.auto import tqdm
from pathlib import Path
import shutil

## Create text file containing all files available

In [2]:
def get_file_list(persist_path):
    root = Path("/glade/collections/cmip/CMIP6")
    dirs = [x for x in root.iterdir() if x.is_dir()]
    for directory in tqdm(dirs):
        print(directory)
        stem = directory.stem
        f = open(f"{persist_path}/{stem}.txt", "w")
        cmd = ["find", "-L", directory.as_posix(), "-name", "*.nc"]
        p = subprocess.Popen(cmd, stderr=subprocess.PIPE, stdout=f)
        p.wait()
        

In [3]:
persist_path = "./CMIP6_filelist"
#get_file_list(persist_path)


## Extract attributes of a file using information from CMI6 DRS.


References
 1. CMIP6 DRS: http://goo.gl/v1drZl
 2. Controlled Vocabularies (CVs) for use in CMIP6:
    https://github.com/WCRP-CMIP/CMIP6_CVs
    
    
Directory structure =
```<mip_era>/
    <activity_id>/
        <institution_id>/
            <source_id>/
                <experiment_id>/
                    <member_id>/
                        <table_id>/
                            <variable_id>/
                                <grid_label>/
                                    <version>
```
file name =
```<variable_id>_<table_id>_<source_id>_<experiment_id >_<member_id>_<grid_label>[_<time_range>].nc```
For time-invariant fields, the last segment (time_range) above is omitted.
Example when there is no sub-experiment: `tas_Amon_GFDL-CM4_historical_r1i1p1f1_gn_196001-199912.nc`
Example with a sub-experiment:   `pr_day_CNRM-CM6-1_dcppA-hindcast_s1960-r2i1p1f1_gn_198001-198412.nc`


In [4]:
activity_ids = list(Path(persist_path).rglob("*.txt"))
activity_ids = [activity_id.stem for activity_id in activity_ids]
activity_ids

['CFMIP',
 'CMIP',
 'LUMIP',
 'LS3MIP',
 'OMIP',
 'HighResMIP',
 'DCPP',
 'AerChemMIP',
 'PAMIP',
 'ScenarioMIP']

In [5]:
df = dd.read_csv(f"{persist_path}/*.txt", header=None).compute()
df.columns = ["path"]
df.head()

,path
0,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
1,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
2,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
3,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
4,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...


In [6]:
len(df)

1027617

In [7]:
def _reverse_filename_format(file_basename, filename_template=None, gridspec_template=None):
    """
    Uses intake's ``reverse_format`` utility to reverse the string method format.
    Given format_string and resolved_string, find arguments
    that would give format_string.format(arguments) == resolved_string
    """
    try:
        return reverse_format(filename_template, file_basename)
    except ValueError:
        try:
            return reverse_format(gridspec_template, file_basename)
        except:
            print(
                f'Failed to parse file: {file_basename} using patterns: {filename_template} and {gridspec_template}'
            )
            return {}
            
def _extract_attr_with_regex(input_str, regex, strip_chars=None):
    pattern = re.compile(regex, re.IGNORECASE)
    match = re.findall(pattern, input_str)
    if match:
        match = max(match, key=len)
        if strip_chars:
            match = match.strip(strip_chars)

        else:
            match = match.strip()

        return match

    else:
        return None
    

exclude_patterns = ['*/files/*', '*/latest/*']
def _filter_func(path):
    return not any(
        fnmatch.fnmatch(path, pat=exclude_pattern) for exclude_pattern in exclude_patterns
    )



In [8]:
%%time
files = df.path.tolist()
filelist = list(filter(_filter_func, files))

CPU times: user 3.66 s, sys: 13.2 ms, total: 3.67 s
Wall time: 3.68 s


In [9]:
len(filelist)

609911

In [10]:
def get_attrs(filepath):
    basename = os.path.basename(filepath)
    dirname = os.path.dirname(filepath)
    filename_template = '{variable_id}_{table_id}_{source_id}_{experiment_id}_{member_id}_{grid_label}_{time_range}.nc'

    gridspec_template = (
                '{variable_id}_{table_id}_{source_id}_{experiment_id}_{member_id}_{grid_label}.nc'
            )
    
    f = _reverse_filename_format(
            basename, filename_template=filename_template, gridspec_template=gridspec_template
        )

    fileparts = {}
    fileparts.update(f)
    parent = os.path.dirname(filepath).strip('/')
    parent_split = parent.split(f"/{fileparts['source_id']}/")
    part_1 = parent_split[0].strip('/').split('/')
    grid_label = parent.split(f"/{fileparts['variable_id']}/")[1].strip('/').split('/')[0]
    fileparts['grid_label'] = grid_label
    fileparts['activity_id'] = part_1[-2]
    fileparts['institution_id'] = part_1[-1]
    version_regex = r'v\d{4}\d{2}\d{2}|v\d{1}'
    version = _extract_attr_with_regex(parent, regex=version_regex) or 'v0'
    fileparts['version'] = version
    fileparts['path'] = filepath
    return fileparts 

In [11]:
%%time
entries = list(map(get_attrs, filelist))

CPU times: user 15.4 s, sys: 622 ms, total: 16.1 s
Wall time: 16.1 s


In [12]:
entries[0]

{'variable_id': 'hfls',
 'table_id': 'Amon',
 'source_id': 'BCC-ESM1',
 'experiment_id': 'ssp370',
 'member_id': 'r2i1p1f1',
 'grid_label': 'gn',
 'time_range': '201501-205512',
 'activity_id': 'AerChemMIP',
 'institution_id': 'BCC',
 'version': 'v20190624',
 'path': '/glade/collections/cmip/CMIP6/AerChemMIP/BCC/BCC-ESM1/ssp370/r2i1p1f1/Amon/hfls/gn/v20190624/hfls/hfls_Amon_BCC-ESM1_ssp370_r2i1p1f1_gn_201501-205512.nc'}

In [13]:
len(entries)

609911

In [14]:
df = pd.DataFrame(entries)
df = df.drop_duplicates(subset=['path'], keep='last').reset_index(drop=True)
df.head()

,variable_id,table_id,source_id,experiment_id,member_id,grid_label,time_range,activity_id,institution_id,version,path
0,hfls,Amon,BCC-ESM1,ssp370,r2i1p1f1,gn,201501-205512,AerChemMIP,BCC,v20190624,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
1,va,Amon,BCC-ESM1,ssp370,r2i1p1f1,gn,201501-205512,AerChemMIP,BCC,v20190624,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
2,tas,Amon,BCC-ESM1,ssp370,r2i1p1f1,gn,201501-205512,AerChemMIP,BCC,v20190624,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
3,rsds,Amon,BCC-ESM1,ssp370,r2i1p1f1,gn,201501-205512,AerChemMIP,BCC,v20190624,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
4,pr,Amon,BCC-ESM1,ssp370,r2i1p1f1,gn,201501-205512,AerChemMIP,BCC,v20190624,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...


In [15]:
len(df)

609911

In [16]:
invalids = df[~df.activity_id.isin(activity_ids)]
df = df[df.activity_id.isin(activity_ids)]

In [17]:
invalids

,variable_id,table_id,source_id,experiment_id,member_id,grid_label,time_range,activity_id,institution_id,version,path
507544,sftof,Ofx,historical,NorESM2-LM,r1i1p1f1,gn,NaN,NCC,NorESM2-LM,v20190815,/glade/collections/cmip/CMIP6/CMIP/NCC/NorESM2...
507546,basin,Ofx,historical,NorESM2-LM,r1i1p1f1,gn,NaN,NCC,NorESM2-LM,v20190815,/glade/collections/cmip/CMIP6/CMIP/NCC/NorESM2...
507549,volcello,Ofx,historical,NorESM2-LM,r1i1p1f1,gr,NaN,NCC,NorESM2-LM,v20190815,/glade/collections/cmip/CMIP6/CMIP/NCC/NorESM2...
508393,areacello,Ofx,piControl,NorESM2-LM,r1i1p1f1,gn,NaN,NCC,NorESM2-LM,v20190815,/glade/collections/cmip/CMIP6/CMIP/NCC/NorESM2...
508395,basin,Ofx,piControl,NorESM2-LM,r1i1p1f1,gn,NaN,NCC,NorESM2-LM,v20190815,/glade/collections/cmip/CMIP6/CMIP/NCC/NorESM2...
508396,volcello,Ofx,piControl,NorESM2-LM,r1i1p1f1,gr,NaN,NCC,NorESM2-LM,v20190815,/glade/collections/cmip/CMIP6/CMIP/NCC/NorESM2...
509418,thetao,Omon,PCMDI-test-1-0,piControl-withism,r3i1p1f1,gn,016201-016201,v20190926,thetao,v20190926,/glade/collections/cmip/CMIP6/CMIP/FIO-QLNM/FI...


In [18]:
len(df)

609904

## Separate Decadal Predictions from the rest of activities

- Decadal prediction catalog requires additional columns (`start_year`)

In [19]:
dcpp = df[df.activity_id=="DCPP"].copy().reset_index(drop=True)
rest = df[~(df.activity_id=="DCPP")].copy().reset_index(drop=True)

In [20]:
columns = ["activity_id", "institution_id", "source_id", "experiment_id", "member_id", "table_id", "variable_id",
           "grid_label", "version", "time_range", "path"]
rest = rest[columns]
rest.head()

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,version,time_range,path
0,AerChemMIP,BCC,BCC-ESM1,ssp370,r2i1p1f1,Amon,hfls,gn,v20190624,201501-205512,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
1,AerChemMIP,BCC,BCC-ESM1,ssp370,r2i1p1f1,Amon,va,gn,v20190624,201501-205512,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
2,AerChemMIP,BCC,BCC-ESM1,ssp370,r2i1p1f1,Amon,tas,gn,v20190624,201501-205512,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
3,AerChemMIP,BCC,BCC-ESM1,ssp370,r2i1p1f1,Amon,rsds,gn,v20190624,201501-205512,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
4,AerChemMIP,BCC,BCC-ESM1,ssp370,r2i1p1f1,Amon,pr,gn,v20190624,201501-205512,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...


In [21]:
rest.to_csv("/glade/collections/cmip/catalog/glade-cmip6.csv.gz", compression="gzip", index=False)
rest.to_csv("../catalogs/glade-cmip6.csv.gz", compression="gzip", index=False)

In [22]:
dcpp.head()

,variable_id,table_id,source_id,experiment_id,member_id,grid_label,time_range,activity_id,institution_id,version,path
0,tas,day,CanESM5,dcppA-hindcast,s2015-r7i1p2f1,gn,20160101-20251231,DCPP,CCCma,v20190429,/glade/collections/cmip/CMIP6/DCPP/CCCma/CanES...
1,tas,day,CanESM5,dcppA-hindcast,s1977-r2i1p2f1,gn,19780101-19871231,DCPP,CCCma,v20190429,/glade/collections/cmip/CMIP6/DCPP/CCCma/CanES...
2,tas,Amon,CanESM5,dcppA-hindcast,s1977-r2i1p2f1,gn,197801-198712,DCPP,CCCma,v20190429,/glade/collections/cmip/CMIP6/DCPP/CCCma/CanES...
3,tas,day,CanESM5,dcppA-hindcast,s1975-r8i1p2f1,gn,19760101-19851231,DCPP,CCCma,v20190429,/glade/collections/cmip/CMIP6/DCPP/CCCma/CanES...
4,tas,Amon,CanESM5,dcppA-hindcast,s1975-r8i1p2f1,gn,197601-198512,DCPP,CCCma,v20190429,/glade/collections/cmip/CMIP6/DCPP/CCCma/CanES...


In [23]:
dcpp["start_year"] = dcpp.member_id.map(lambda x: x.split("-")[0][1:])
dcpp["member_id"] = dcpp["member_id"].map(lambda x: x.split("-")[-1])
dcpp.head()

,variable_id,table_id,source_id,experiment_id,member_id,grid_label,time_range,activity_id,institution_id,version,path,start_year
0,tas,day,CanESM5,dcppA-hindcast,r7i1p2f1,gn,20160101-20251231,DCPP,CCCma,v20190429,/glade/collections/cmip/CMIP6/DCPP/CCCma/CanES...,2015
1,tas,day,CanESM5,dcppA-hindcast,r2i1p2f1,gn,19780101-19871231,DCPP,CCCma,v20190429,/glade/collections/cmip/CMIP6/DCPP/CCCma/CanES...,1977
2,tas,Amon,CanESM5,dcppA-hindcast,r2i1p2f1,gn,197801-198712,DCPP,CCCma,v20190429,/glade/collections/cmip/CMIP6/DCPP/CCCma/CanES...,1977
3,tas,day,CanESM5,dcppA-hindcast,r8i1p2f1,gn,19760101-19851231,DCPP,CCCma,v20190429,/glade/collections/cmip/CMIP6/DCPP/CCCma/CanES...,1975
4,tas,Amon,CanESM5,dcppA-hindcast,r8i1p2f1,gn,197601-198512,DCPP,CCCma,v20190429,/glade/collections/cmip/CMIP6/DCPP/CCCma/CanES...,1975


In [24]:
columns = ["activity_id", "institution_id", "source_id", "experiment_id", "member_id", "start_year", "table_id", "variable_id",
           "grid_label", "version", "time_range", "path"]
dcpp = dcpp[columns]
dcpp.head()

,activity_id,institution_id,source_id,experiment_id,member_id,start_year,table_id,variable_id,grid_label,version,time_range,path
0,DCPP,CCCma,CanESM5,dcppA-hindcast,r7i1p2f1,2015,day,tas,gn,v20190429,20160101-20251231,/glade/collections/cmip/CMIP6/DCPP/CCCma/CanES...
1,DCPP,CCCma,CanESM5,dcppA-hindcast,r2i1p2f1,1977,day,tas,gn,v20190429,19780101-19871231,/glade/collections/cmip/CMIP6/DCPP/CCCma/CanES...
2,DCPP,CCCma,CanESM5,dcppA-hindcast,r2i1p2f1,1977,Amon,tas,gn,v20190429,197801-198712,/glade/collections/cmip/CMIP6/DCPP/CCCma/CanES...
3,DCPP,CCCma,CanESM5,dcppA-hindcast,r8i1p2f1,1975,day,tas,gn,v20190429,19760101-19851231,/glade/collections/cmip/CMIP6/DCPP/CCCma/CanES...
4,DCPP,CCCma,CanESM5,dcppA-hindcast,r8i1p2f1,1975,Amon,tas,gn,v20190429,197601-198512,/glade/collections/cmip/CMIP6/DCPP/CCCma/CanES...


In [25]:
dcpp.to_csv("/glade/collections/cmip/catalog/glade-cmip6-dcpp.csv.gz", compression="gzip", index=False)
dcpp.to_csv("../catalogs/glade-cmip6-dcpp.csv.gz", compression="gzip", index=False)